<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/main/Fall_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and frameworks

In [1]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200


Get the sisfall and sisfall_enhanced dataset

In [ ]:
!gdown --id 1kyTRhIFhqwRkf9gERof1Xm5FVQ-klLVA
!gdown --id 1gvOuxPc8dNgTnxuvPcVuCKifOf98-TV0
!unzip SisFall_dataset.zip
!unzip SisFall_enhanced.zip

# Implementation

Get all addresses

In [ ]:
def get_file_name(path):
  allfiles = []
  allFolders = glob.glob(path + "*")
  for files in allFolders:
      allfiles.append(glob.glob(files+"/*.txt"))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')
  return np.hstack(allfiles)

In [ ]:
get_file_name("SisFall_dataset/")

array(['SisFall_dataset/SA14/F11_SA14_R04.txt',
       'SisFall_dataset/SA14/F05_SA14_R03.txt',
       'SisFall_dataset/SA14/D13_SA14_R05.txt', ...,
       'SisFall_dataset/SA15/F05_SA15_R02.txt',
       'SisFall_dataset/SA15/D16_SA15_R02.txt',
       'SisFall_dataset/SA15/D13_SA15_R02.txt'], dtype='<U37')

read dataset from address path

In [ ]:
def read_data(data_path):
    data = pd.read_csv(data_path, header=None)
    data.columns = ['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z', 'MMA8451Q_x',
                    'MMA8451Q_y', 'MMA8451Q_z']
    data['MMA8451Q_z'] = data['MMA8451Q_z'].map(lambda x: str(x)[:-1])
    for name in data.columns :
      data[name] = data[name].astype(float)
    return data

Feature Extraction

In [ ]:
def add_features(dataset,data_path):
    new_dataset = pd.DataFrame()
    new_dataset['acc_1'] = dataset.apply(
        lambda row: sqrt((row.ADXL345_x ** 2 + row.ADXL345_y ** 2 + row.ADXL345_z ** 2)), axis=1)
    new_dataset['acc_2'] = dataset.apply(
        lambda row: sqrt((row.MMA8451Q_x ** 2 + row.MMA8451Q_y ** 2 + row.MMA8451Q_z ** 2)), axis=1)
    new_dataset['geo'] = dataset.apply(
        lambda row: sqrt((row.ITG3200_x ** 2 + row.ITG3200_y ** 2 + row.ITG3200_z ** 2)), axis=1)
    new_dataset['label'] = get_label(data_path)
    return np.round(new_dataset.to_numpy(),2)


Get the Label

In [ ]:
def get_label(data_path):
    label = data_path[21]
    if label =='D':
      return int(0)
    elif label =='F':  
      label_path = data_path.replace('dataset','enhanced')
      labels = pd.read_csv(label_path,header=None)
      labels[labels == 2] = 1
      return labels


Split Dataset to train and test

In [ ]:
def split_address(dataset_address):
  np.random.shuffle(dataset_address)
  train, test = np.split(dataset_address, [int(len(dataset_address)*0.7)])
  return train, test

Extract features from All addresses

In [4]:
def datasets_to_nparray(datasets_address_array):
  result = np.empty((0, 4), int)
  for address in datasets_address_array:
    result = np.concatenate(
        (result,add_features(read_data(address),address)),axis=0)
  return result

Windowing of the dataset

In [ ]:
def windowing(dataset,window_size):
  window = window_size * (dataset.shape[1]-1)
  cut = dataset.shape[0] % window_size
  feature = dataset[:-cut,0:-1]
  label = dataset[:-cut,-1]
  feature = feature.ravel().reshape(feature.size//window,window)
  label = label.reshape(label.size//window_size,window_size)
  label = label.sum(axis=1)
  label[label > 0] = 1
  return feature,label

Save the Train and Test Dataset as Tensor

In [ ]:
def dataset_to_tensor(test,train,window_size):
  test_feature , test_label = windowing(datasets_to_nparray(test),window_size)
  np.savez('Sisfall_data_test', inputs=test_feature, targets=test_label)
  train_feature , train_label = windowing(datasets_to_nparray(train),window_size)
  np.savez('Sisfall_data_train', inputs=train_feature, targets=train_label)
